# MVP de Engenharia de Dados: Análise de Níveis de Obesidade
Objetivo: Desenvolver um pipeline de dados para analisar a relação entre hábitos alimentares, atividade física e níveis de obesidade, utilizando um modelo estrela em Databricks.

## 1. Objetivo do Trabalho


### 1.1 **Problema**  
A obesidade é um dos principais fatores de risco para doenças crônicas como diabetes tipo 2, hipertensão e doenças cardiovasculares. Por isso, é fundamental compreender como hábitos alimentares, fatores demográficos e o nível de atividade física influenciam o grau de obesidade dos indivíduos.  
A base de dados utilizada é oriunda de uma pesquisa com informações sobre hábitos alimentares e condições físicas de indivíduos de México, Peru e Colômbia. Ela contém 17 atributos e 2.111 registros, sendo o atributo NObeyesdad o responsável por categorizar o nível de obesidade dos participantes.  
A partir disso, foram elaboradas as seguintes perguntas-chave a serem respondidas com base na análise dos dados:
- **1-** Pessoas que consomem mais água tendem a ter menor nível de obesidade?
- **2-** Qual a distribuição de obesidade por gênero e faixa etária?
- **3-** Existe uma correlação entre altura, peso e nível de obesidade?
- **4-** Qual a média de idade das pessoas em cada nível de obesidade?
- **5-** Quem come mais vegetais tem menor índice de obesidade?
- **6-** Qual o impacto do consumo frequente de alimentos calóricos na obesidade?
- **7-** Pessoas que fazem mais atividades físicas têm menor nível de obesidade?
- **8-** Existe uma relação entre hábitos saudáveis (maior consumo de vegetais, menor consumo de alimentos calóricos e maior atividade física) e os níveis de obesidade?
- **9-** Pessoas que comem mais refeições por dia têm menor tendência à obesidade?
- **10-** Há uma relação entre o consumo de álcool e os níveis de obesidade?
- **11-** Quem usa transporte ativo (bicicleta, caminhada) tem menores índices de obesidade do que aqueles que usam carro ou transporte público?



### 1.2 **Solução Proposta**  
Este projeto propõe a construção de uma pipeline de dados completa, estruturada em três camadas (Bronze → Silver → Gold), que permite organizar, transformar e analisar os dados de forma eficiente, modular e escalável. A arquitetura escolhida também possibilita uma separação clara entre dados brutos, dados limpos e dados prontos para análise, facilitando a rastreabilidade e reuso.

## 2. Base de Dados Escolhida


Dataset: [Obesity Levels Prediction](https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition)

A base de dados tem os seguintes atributos:  
- Gender - Categórico - Male/Female - Genero do indivíduo.
- Age - Contínuo - Idade do indivíduo.
- Height - Contínuo - Altura do indivíduo.
- Weight - Contínuo - Peso do indivíduo.
- family_history_with_overweight - Binário - Algum membro da família do indivíduo já sofreu ou sofre de sobrepeso?
- FAVC - Binário - O indivíduo consome alimentos altamente calóricos com frequência?
- FCVC - Inteiro - O indivíduo costuma comer vegetais nas suas refeições?
- NCP - Contínuo - Quantas refeições principais o indivíduo faz diariamente?
- CAEC - Categórico - no/Sometimes/Frequently/Always - O indivíduo consome algum alimento entre as refeições?
- SMOKE - Binário - O indivíduo fuma?
- CH2O - Contínuo - Quanto de água o indivíduo bebe diariamente?
- SCC - Binário - O indivíduo monitora as calorias que consome diariamente?
- FAF - Contínuo - Com que frequência o indivíduo pratica atividade física?
- TUE - Inteiro - Quanto tempo o indivíduo utiliza dispositivos tecnológicos, como celular, videogames, televisão, computador e outros?
- CALC - Categórico - no/Sometimes/Frequently/Always - Com que frequência o indivíduo consome álcool?
- MTRANS - Categórico - Public_Transportation,Walking,Automobile,Motorbike,Bike - Qual meio de transporte o indivíduo costuma usar?
- NObeyesdad - Categórico - Insufficient_Weight/Normal_Weight/Overweight_Level_I/Overweight_Level_II/Obesity_Type_I/Obesity_Type_II/Obesity_Type_III - Nível de obesidade do indivíduo

Como o dataset utilizado é público e foi previamente anonimizado, não contendo informações pessoais identificáveis, consideramos que cada linha da tabela é um paciente entrevistado pela pesquisa, logo não podemos ter linhas duplicadas.

## 3. Coleta dos Dados


A etapa de coleta de dados consiste em armazenar e disponibilizar o conjunto de dados para o início do processamento da pipeline de dados.  
O arquivo CSV foi armazenado no GitHub como ponto de acesso centralizado dos dados brutos. A leitura inicial foi feita com a biblioteca Pandas, permitindo a inspeção e visualização preliminar do dataset. Em seguida, foi realizada, via linguagem SQL, a criação do Database da Camada Bronze. O DataFrame criado com o Pandas foi convertido para um DataFrame do Spark, possibilitando o processamento distribuído e eficiente. Por fim, os dados foram persistidos como uma tabela Delta no catálogo, utilizando o comando saveAsTable("bronze.obesity_raw"), dentro do database da camada Bronze, garantindo versionamento e otimização para as etapas seguintes da pipeline.

In [0]:
import pandas as pd
from pyspark.sql import SparkSession

In [0]:
# URL do dataset
url_dados = 'https://raw.githubusercontent.com/geovane186/MVP_Analyse_Obesity_Levels/refs/heads/main/DataSet/ObesityDataSet_raw_and_data_sinthetic.csv'

# Carrega para DataFrame pandas
try:
    obesity_pandas_df = pd.read_csv(url_dados)
    print("✅ Dados carregados com sucesso. Amostra:")
    display(obesity_pandas_df.head(3))
    
except Exception as e:
    print(f"❌ Erro ao carregar dados: {str(e)}")
    dbutils.notebook.exit("Interrompendo execução")

✅ Dados carregados com sucesso. Amostra:


Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
Male,23.0,1.8,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [0]:
%sql DROP DATABASE IF EXISTS  bronze CASCADE

In [0]:
%sql
CREATE DATABASE bronze
COMMENT "Camada raw para dados brutos"
LOCATION "/mnt/bronze";

DROP TABLE IF EXISTS bronze.obesity_raw;

In [0]:
obesity_spark_df = spark.createDataFrame(obesity_pandas_df)
obesity_spark_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("bronze.obesity_raw")

In [0]:
%sql SELECT * FROM bronze.obesity_raw LIMIT 5;

Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
Male,23.0,1.8,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
Male,27.0,1.8,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


## 4. Análise - Verificação de Qualidade


Após o armazenamento na camada Bronze, os dados passaram por uma verificação de qualidade com foco em três pontos principais: **valores nulos**, **valores inválidos** e **registros duplicados**. A verificação foi feita com comandos SQL, onde cada coluna foi avaliada quanto à presença de valores nulos, diversidade de valores, percentual de dados válidos conforme regras definidas (por exemplo: faixas aceitáveis de idade, altura e peso; valores válidos para variáveis categóricas como gênero e hábitos). Não foram encontrados valores nulos ou inválidos.

Em seguida, foi realizada uma verificação de duplicatas, agrupando todos os campos para identificar registros exatamente iguais. Foram identificadas 24 ocorrências duplicadas, que serão removidas na criação da tabela Silver, garantindo que cada registro represente um indivíduo único. Essa foi a única inconsistência encontrada nos dados da camada Bronze.

In [0]:
%sql
WITH column_stats AS (
  -- Gender
  SELECT
    'Gender' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT Gender) AS distinct_values,
    SUM(CASE WHEN Gender IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN Gender IN ('Male', 'Female') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(Gender) AS sample_values,
    'Valores válidos: Male, Female' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- Age
  SELECT
    'Age' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT Age) AS distinct_values,
    SUM(CASE WHEN Age IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN Age BETWEEN 14 AND 100 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(Age), MAX(Age)) AS sample_values,
    'Intervalo válido: 14-100 anos' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- Height
  SELECT
    'Height' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT Height) AS distinct_values,
    SUM(CASE WHEN Height IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN Height BETWEEN 1.2 AND 2.5 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(Height), MAX(Height)) AS sample_values,
    'Intervalo válido: 1.2m - 2.5m' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- Weight
  SELECT
    'Weight' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT Weight) AS distinct_values,
    SUM(CASE WHEN Weight IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN Weight BETWEEN 30 AND 300 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(Weight), MAX(Weight)) AS sample_values,
    'Intervalo válido: 30kg - 300kg' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- family_history_with_overweight
  SELECT
    'family_history_with_overweight' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT family_history_with_overweight) AS distinct_values,
    SUM(CASE WHEN family_history_with_overweight IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN family_history_with_overweight IN ('yes', 'no') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(family_history_with_overweight) AS sample_values,
    'Valores válidos: yes, no' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- FAVC
  SELECT
    'FAVC' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT FAVC) AS distinct_values,
    SUM(CASE WHEN FAVC IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN FAVC IN ('yes', 'no') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(FAVC) AS sample_values,
    'Valores válidos: yes, no' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- FCVC
  SELECT
    'FCVC' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT FCVC) AS distinct_values,
    SUM(CASE WHEN FCVC IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN FCVC BETWEEN 1 AND 3 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(FCVC), MAX(FCVC)) AS sample_values,
    'Intervalo válido: 1-3 (escala Likert)' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- NCP
  SELECT
    'NCP' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT NCP) AS distinct_values,
    SUM(CASE WHEN NCP IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN NCP BETWEEN 1 AND 4 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(NCP), MAX(NCP)) AS sample_values,
    'Intervalo válido: 1-4 refeições diárias' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- CAEC
  SELECT
    'CAEC' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT CAEC) AS distinct_values,
    SUM(CASE WHEN CAEC IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN CAEC IN ('no', 'Sometimes', 'Frequently', 'Always') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(CAEC) AS sample_values,
    'Valores válidos: no, Sometimes, Frequently, Always' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- SMOKE
  SELECT
    'SMOKE' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT SMOKE) AS distinct_values,
    SUM(CASE WHEN SMOKE IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN SMOKE IN ('yes', 'no') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(SMOKE) AS sample_values,
    'Valores válidos: yes, no' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- CH2O
  SELECT
    'CH2O' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT CH2O) AS distinct_values,
    SUM(CASE WHEN CH2O IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN CH2O BETWEEN 1 AND 3 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(CH2O), MAX(CH2O)) AS sample_values,
    'Intervalo válido: 1-3 (escala Likert)' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- SCC
  SELECT
    'SCC' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT SCC) AS distinct_values,
    SUM(CASE WHEN SCC IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN SCC IN ('yes', 'no') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(SCC) AS sample_values,
    'Valores válidos: yes, no' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- FAF
  SELECT
    'FAF' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT FAF) AS distinct_values,
    SUM(CASE WHEN FAF IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN FAF BETWEEN 0 AND 3 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(FAF), MAX(FAF)) AS sample_values,
    'Intervalo válido: 0-3 (escala Likert)' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- TUE
  SELECT
    'TUE' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT TUE) AS distinct_values,
    SUM(CASE WHEN TUE IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN TUE BETWEEN 0 AND 2 THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    ARRAY(MIN(TUE), MAX(TUE)) AS sample_values,
    'Intervalo válido: 0-2 horas diárias' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- CALC
  SELECT
    'CALC' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT CALC) AS distinct_values,
    SUM(CASE WHEN CALC IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN CALC IN ('no', 'Sometimes', 'Frequently', 'Always') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(CALC) AS sample_values,
    'Valores válidos: no, Sometimes, Frequently, Always' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- MTRANS
  SELECT
    'MTRANS' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT MTRANS) AS distinct_values,
    SUM(CASE WHEN MTRANS IS NULL THEN 1 ELSE 0 END) AS null_count,
    ROUND(AVG(CASE WHEN MTRANS IN ('Public_Transportation', 'Walking', 'Automobile', 'Motorbike', 'Bike') THEN 1 ELSE 0 END) * 100, 2) AS validity_percentage,
    COLLECT_SET(MTRANS) AS sample_values,
    'Valores válidos: Public_Transportation, Walking, Automobile, Motorbike, Bike' AS validation_rule
  FROM bronze.obesity_raw
  
  UNION ALL
  
  -- NObeyesdad
  SELECT
    'NObeyesdad' AS column_name,
    COUNT(*) AS total_rows,
    COUNT(DISTINCT NObeyesdad) AS distinct_values,
    SUM(CASE WHEN NObeyesdad IS NULL THEN 1 ELSE 0 END) AS null_count,
    100 AS validity_percentage, -- Assume-se que todos os valores são válidos
    COLLECT_SET(NObeyesdad) AS sample_values,
    'Valores válidos: Insufficient_Weight, Normal_Weight, Overweight_Level_I, Overweight_Level_II, Obesity_Type_I, Obesity_Type_II, Obesity_Type_III' AS validation_rule
  FROM bronze.obesity_raw
)

SELECT
  column_name,
  total_rows,
  null_count,
  ROUND((null_count * 100.0 / total_rows), 2) AS null_percentage,
  distinct_values,
  validity_percentage,
  CASE
    WHEN null_percentage > 5 THEN '❌ Alta taxa de nulos'
    WHEN validity_percentage < 100 THEN '⚠️ Valores inválidos'
    ELSE '✅ OK'
  END AS status,
  validation_rule,
  sample_values
FROM column_stats
ORDER BY 
  CASE 
    WHEN status LIKE '❌%' THEN 0
    WHEN status LIKE '⚠️%' THEN 1
    ELSE 2
  END,
  column_name;

column_name,total_rows,null_count,null_percentage,distinct_values,validity_percentage,status,validation_rule,sample_values
Age,2111,0,0.00,1402,100.0,✅ OK,Intervalo válido: 14-100 anos,"List(14.0, 61.0)"
CAEC,2111,0,0.00,4,100.0,✅ OK,"Valores válidos: no, Sometimes, Frequently, Always","List(no, Always, Sometimes, Frequently)"
CALC,2111,0,0.00,4,100.0,✅ OK,"Valores válidos: no, Sometimes, Frequently, Always","List(no, Always, Sometimes, Frequently)"
CH2O,2111,0,0.00,1268,100.0,✅ OK,Intervalo válido: 1-3 (escala Likert),"List(1.0, 3.0)"
FAF,2111,0,0.00,1190,100.0,✅ OK,Intervalo válido: 0-3 (escala Likert),"List(0.0, 3.0)"
FAVC,2111,0,0.00,2,100.0,✅ OK,"Valores válidos: yes, no","List(no, yes)"
FCVC,2111,0,0.00,810,100.0,✅ OK,Intervalo válido: 1-3 (escala Likert),"List(1.0, 3.0)"
Gender,2111,0,0.00,2,100.0,✅ OK,"Valores válidos: Male, Female","List(Female, Male)"
Height,2111,0,0.00,1574,100.0,✅ OK,Intervalo válido: 1.2m - 2.5m,"List(1.45, 1.98)"
MTRANS,2111,0,0.00,5,100.0,✅ OK,"Valores válidos: Public_Transportation, Walking, Automobile, Motorbike, Bike","List(Bike, Walking, Public_Transportation, Motorbike, Automobile)"


In [0]:
%sql
SELECT 
  Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad, COUNT(*) as record_count
FROM bronze.obesity_raw
GROUP BY Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
HAVING COUNT(*) > 1
ORDER BY record_count DESC;


Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,record_count
Male,21.0,1.62,70.0,no,yes,2.0,1.0,no,no,3.0,no,1.0,0.0,Sometimes,Public_Transportation,Overweight_Level_I,15
Female,21.0,1.52,42.0,no,yes,3.0,1.0,Frequently,no,1.0,no,0.0,0.0,Sometimes,Public_Transportation,Insufficient_Weight,4
Male,18.0,1.72,53.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,2.0,Sometimes,Public_Transportation,Insufficient_Weight,2
Female,21.0,1.52,42.0,no,no,3.0,1.0,Frequently,no,1.0,no,0.0,0.0,Sometimes,Public_Transportation,Insufficient_Weight,2
Female,22.0,1.69,65.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Sometimes,Public_Transportation,Normal_Weight,2
Female,25.0,1.57,55.0,no,yes,2.0,1.0,Sometimes,no,2.0,no,2.0,0.0,Sometimes,Public_Transportation,Normal_Weight,2
Female,16.0,1.66,58.0,no,no,2.0,1.0,Sometimes,no,1.0,no,0.0,1.0,no,Walking,Normal_Weight,2
Female,18.0,1.62,55.0,yes,yes,2.0,3.0,Frequently,no,1.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight,2
Male,22.0,1.74,75.0,yes,yes,3.0,3.0,Frequently,no,1.0,no,1.0,0.0,no,Automobile,Normal_Weight,2


In [0]:
%sql
WITH duplicated_rows AS (
  SELECT 
    Gender, Age, Height, Weight, family_history_with_overweight, FAVC, FCVC, NCP, CAEC,
    SMOKE, CH2O, SCC, FAF, TUE, CALC, MTRANS, NObeyesdad,
    COUNT(*) AS record_count
  FROM bronze.obesity_raw
  GROUP BY Gender, Age, Height, Weight, family_history_with_overweight, FAVC, FCVC, NCP, CAEC,
           SMOKE, CH2O, SCC, FAF, TUE, CALC, MTRANS, NObeyesdad
  HAVING COUNT(*) > 1
)
SELECT 
  SUM(record_count - 1) AS total_duplicated_records
FROM duplicated_rows;

total_duplicated_records
24


## 5. Carga e Modelagem (Modelo Estrela)

### 5.1 Camada Silver (Clean)

Inicialmente para criar a camada silver criamos o Database, e em seguida construimos a tabela silver.obesity_clean a partir da tabela bronze.obesity_raw, aplicando filtros, limpeza, padronização e transformações semânticas. O objetivo foi criar uma estrutura mais confiável e analiticamente útil.

**Principais etapas realizadas:**  
- Remoção de registros duplicados:
  - Foi gerado o identificador **patient_key** por meio de uma função hash (MD5) aplicada à concatenação de todos os campos.
  - Com o uso de **SELECT DISTINCT**, garantiu-se que cada registro fosse único, removendo os 9 registros duplicados identificados previamente.
- Validação de Faixas Válidas:
  - Aplicados filtros em **Age**, **Height** e **Weight** com base nos critérios definidos na verificação de qualidade.
- Padronização de Dados:
  - **Gender** foi convertido para letras maiúsculas e limpo.
  - Campos categóricos como **family_history_with_overweight**, **FAVC**, **SMOKE**, **SCC**, etc., foram transformados em valores booleanos ou categorias normalizadas.
  - Categorias como **CAEC**, **CALC**, e **MTRANS** foram reclassificadas em grupos mais analíticos (por exemplo, active, car, public_transport para o transporte).
- Variáveis Derivadas:
  - **age_group**: segmentação por faixa etária.
  - **bmi**: cálculo do índice de massa corporal (IMC).
  - **obesity_category**: classificação geral em underweight, normal, overweight, e obese, com base no campo NObeyesdad.
- Adição de Metadados:
  - A coluna **processed_at** registra o timestamp da transformação para fins de rastreabilidade.

Ao fim da criação fizemos uma verificação de duplicatas na tabela criada, e não encontramos mais problemas.



In [0]:
%sql DROP DATABASE IF EXISTS  silver CASCADE

In [0]:
%sql
CREATE DATABASE silver
COMMENT "Dados limpos e enriquecidos com regras de qualidade aplicadas"
LOCATION "/mnt/silver";

In [0]:
%sql
-- Criação da tabela silver com remoção de duplicatas exatas
CREATE TABLE IF NOT EXISTS silver.obesity_clean AS
WITH base AS (
  SELECT
    MD5(CONCAT_WS('|', 
      TRIM(Gender), Age, Height, Weight, 
      family_history_with_overweight, FAVC, FCVC, NCP, CAEC, 
      SMOKE, CH2O, SCC, FAF, TUE, CALC, MTRANS, NObeyesdad
    )) AS patient_key,
    *
    
  FROM bronze.obesity_raw
  WHERE 
    Age BETWEEN 14 AND 100 
    AND Height BETWEEN 1.2 AND 2.5 
    AND Weight BETWEEN 30 AND 300
)
SELECT DISTINCT -- Remove duplicatas automaticamente com base no patient_key
  patient_key,
  UPPER(TRIM(Gender)) AS gender,
  CAST(Age AS INT) AS age,

  -- Faixa Etária
  CASE
    WHEN age < 18 THEN 'adolescent'
    WHEN age BETWEEN 18 AND 30 THEN 'young_adult'
    WHEN age BETWEEN 31 AND 50 THEN 'adult'
    ELSE 'senior'
  END AS age_group,

  -- Biometria
  CAST(Height AS DECIMAL(10,4)) AS height_m,
  CAST(Weight AS DECIMAL(10,4)) AS weight_kg,
  ROUND(CAST(Weight AS DOUBLE) / POWER(CAST(Height AS DOUBLE), 2), 1) AS bmi,

  -- Histórico familiar
  CASE WHEN family_history_with_overweight = 'yes' THEN TRUE ELSE FALSE END AS has_family_history,

  -- Hábitos alimentares
  CASE WHEN FAVC = 'yes' THEN 'high_calorie' ELSE 'low_calorie' END AS calorie_intake_level,
  CAST(FCVC AS DOUBLE) AS vegetable_intake_freq,
  CAST(NCP AS INT) AS meals_per_day,
  -- Campo CAEC (alimento entre refeições)
  CASE
    WHEN CAEC = 'no' THEN 'none'
    WHEN CAEC = 'Sometimes' THEN 'sometimes'
    WHEN CAEC = 'Frequently' THEN 'frequently'
    WHEN CAEC = 'Always' THEN 'always'
    ELSE 'unknown'
  END AS eating_between_meals,

  -- Campo CALC (Consumo de álcool)
  CASE
    WHEN CALC = 'no' THEN 'non_drinker'
    WHEN CALC IN ('Sometimes', 'Frequently', 'Always') THEN LOWER(CALC)
    ELSE 'unknown'
  END AS alcohol_consumption,

  -- Hábitos de saúde
  CASE WHEN SMOKE = 'yes' THEN TRUE ELSE FALSE END AS is_smoker,
  CAST(CH2O AS DOUBLE) AS water_intake,
  CASE WHEN SCC = 'yes' THEN TRUE ELSE FALSE END AS monitors_calories,

  -- Atividade física
  CAST(FAF AS DOUBLE) AS physical_activity_freq,
  CAST(TUE AS DOUBLE) AS screen_time_hours,

  -- Transporte
  CASE
    WHEN MTRANS = 'Walking' THEN 'active'
    WHEN MTRANS = 'Bike' THEN 'active'
    WHEN MTRANS = 'Automobile' THEN 'car'
    WHEN MTRANS = 'Public_Transportation' THEN 'public_transport'
    ELSE 'other'
  END AS transportation_type,

  -- Níveis de obesidade
  TRIM(NObeyesdad) AS obesity_level,
  CASE
    WHEN NObeyesdad = 'Normal_Weight' THEN 'normal'
    WHEN NObeyesdad LIKE 'Overweight%' THEN 'overweight'
    WHEN NObeyesdad LIKE 'Obesity%' THEN 'obese'
    ELSE 'underweight'
  END AS obesity_category,

  -- Metadados
  current_timestamp() AS processed_at
FROM base;

num_affected_rows,num_inserted_rows


In [0]:
%sql SELECT * FROM silver.obesity_clean LIMIT 5;


patient_key,gender,age,age_group,height_m,weight_kg,bmi,has_family_history,calorie_intake_level,vegetable_intake_freq,meals_per_day,eating_between_meals,alcohol_consumption,is_smoker,water_intake,monitors_calories,physical_activity_freq,screen_time_hours,transportation_type,obesity_level,obesity_category,processed_at
fd4d589c0c96dfd91baca2a2ac2d5001,MALE,18,young_adult,1.7081,51.3147,17.6,true,high_calorie,1.303878,3,sometimes,sometimes,false,1.755497,false,0.062932,1.672532,public_transport,Insufficient_Weight,underweight,2025-04-09T02:00:21.666+0000
41c5fd1e097f807578e8021fb1329abd,MALE,21,young_adult,1.7122,98.7000,33.7,true,high_calorie,2.0,2,sometimes,non_drinker,false,2.205977,false,2.698874,2.0,public_transport,Obesity_Type_I,obese,2025-04-09T02:00:21.666+0000
655411e1adb4032e225aa2195c766dd5,FEMALE,23,young_adult,1.6108,82.5330,31.8,true,high_calorie,2.09663,2,sometimes,non_drinker,false,2.432967,false,1.887012,0.0,public_transport,Obesity_Type_I,obese,2025-04-09T02:00:21.666+0000
74d0b8b0a2862482dd5f4a556a208611,MALE,18,young_adult,1.7387,50.2487,16.6,true,high_calorie,1.871213,3,sometimes,sometimes,false,1.283738,false,0.684879,1.487223,public_transport,Insufficient_Weight,underweight,2025-04-09T02:00:21.666+0000
66d6ff722631c2112289d5fc63aacd80,FEMALE,16,adolescent,1.6038,65.0000,25.3,true,high_calorie,2.543563,1,sometimes,sometimes,false,2.0,true,0.694281,1.056911,public_transport,Overweight_Level_I,overweight,2025-04-09T02:00:21.666+0000


In [0]:
%sql
SELECT 
  gender, age, height_m, weight_kg, 
  has_family_history, calorie_intake_level, vegetable_intake_freq,
  meals_per_day, eating_between_meals, is_smoker, water_intake,
  monitors_calories, physical_activity_freq, screen_time_hours,
  alcohol_consumption, transportation_type, obesity_level, patient_key,
  COUNT(*) as count
FROM silver.obesity_clean
GROUP BY 
  gender, age, height_m, weight_kg, 
  has_family_history, calorie_intake_level, vegetable_intake_freq,
  meals_per_day, eating_between_meals, is_smoker, water_intake,
  monitors_calories, physical_activity_freq, screen_time_hours,
  alcohol_consumption, transportation_type, obesity_level,patient_key
HAVING COUNT(*) > 1;

gender,age,height_m,weight_kg,has_family_history,calorie_intake_level,vegetable_intake_freq,meals_per_day,eating_between_meals,is_smoker,water_intake,monitors_calories,physical_activity_freq,screen_time_hours,alcohol_consumption,transportation_type,obesity_level,patient_key,count


### 5.2 Camada Gold (Modelo Estrela)

Neste projeto escolhemos o modelo estrela (star schema) por sua simplicidade, eficiência e clareza na organização dos dados, especialmente voltado para análise e consumo em um ambiente de Data Warehouse. Ele permite uma separação lógica entre:
- Fatos (dados quantitativos e de interesse de análise, como o nível de obesidade);
- Dimensões (dados descritivos que fornecem contexto, como hábitos, atividade física e características dos pacientes).

Além disso esse modelo oferece uma estrutura simples e intuitiva, facilitando a interpretação por analistas e usuários finais. Sua arquitetura com junções por chaves simples proporciona alta performance em consultas analíticas, além de permitir modularidade na manutenção e expansão das dimensões ou inclusão de novas métricas. Essa abordagem também está alinhada às boas práticas de modelagem dimensional propostas por Ralph Kimball, amplamente reconhecidas em ambientes de Data Warehousing.

Inicialmente, criamos o database **gold** para ser utilizado como camada final do pipeline. Em seguida, foram criadas as seguintes tabelas dimensão:

**DIM_PACIENTE**: Dados Demográficos e Biométricos

**DIM_HABITOS**: Hábitos Alimentares e de Consumo

**DIM_ATIVIDADE**: Comportamento Físico, Transporte e Uso de Tecnologia

E a tabela fato:

**FACT_OBESIDADE**: Métricas de obesidade com as referências às dimensões

Posteriormente, realizamos a inserção dos dados transformados da tabela silver.obesity_clean para compor a camada gold. Por fim, foi executada uma verificação final de integridade entre as tabelas.

⚠️ _**Vale ressaltar que, como utilizamos a versão Community do Databricks, o uso de constraints (chaves primárias e estrangeiras) não foi permitido. Por isso, a verificação de integridade foi essencial e as relações entre tabelas serão explicitamente descritas no Catálogo de Dados.**_



In [0]:
%sql DROP DATABASE IF EXISTS gold CASCADE

In [0]:
%sql
CREATE DATABASE gold
COMMENT "Dados para o Modelo Estrela"
LOCATION "/mnt/gold";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_paciente (
  id_paciente STRING NOT NULL COMMENT 'Hash MD5 dos campos naturais',
  gender STRING NOT NULL COMMENT 'Gênero (M/F)',
  age INT NOT NULL COMMENT 'Idade em anos',
  age_group STRING NOT NULL COMMENT 'Faixa etária: adolescent/young_adult/adult/senior',
  height_m DECIMAL(10,4) NOT NULL COMMENT 'Altura em metros',
  weight_kg DECIMAL(10,4) NOT NULL COMMENT 'Peso em kg',
  bmi DECIMAL(4,1) NOT NULL COMMENT 'Índice de Massa Corporal',
  has_family_history BOOLEAN NOT NULL COMMENT 'Informa se algum membro da família do paciente já sofreu ou sofre de sobrepeso'
)
USING DELTA
LOCATION '/mnt/gold/dim_paciente'
COMMENT 'Dimensão com informações demográficas e biométricas dos pacientes';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_habitos (
  id_habitos STRING NOT NULL COMMENT 'Hash MD5 dos hábitos alimentares',
  calorie_intake_level STRING NOT NULL COMMENT 'low_calorie/high_calorie',
  vegetable_intake_freq DOUBLE NOT NULL COMMENT 'Frequência de consumo de vegetais (1-3)',
  meals_per_day INT NOT NULL COMMENT 'Número de refeições principais diárias',
  eating_between_meals STRING NOT NULL COMMENT 'none/sometimes/frequently/always',
  alcohol_consumption STRING NOT NULL COMMENT 'non_drinker/sometimes/frequently/always',
  is_smoker BOOLEAN NOT NULL COMMENT 'Indica se o paciente é fumante',
  water_intake DOUBLE NOT NULL COMMENT 'Frequência de consumo de água (1-3)',
  monitors_calories BOOLEAN NOT NULL COMMENT 'Monitora ingestão calórica'
)
USING DELTA
LOCATION '/mnt/gold/dim_habitos'
COMMENT 'Dimensão com hábitos alimentares e de consumo';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_atividade (
  id_atividade STRING NOT NULL COMMENT 'Hash MD5 das atividades físicas',
  physical_activity_freq DOUBLE NOT NULL COMMENT 'Frequência de atividade física (0-3)',
  transportation_type STRING NOT NULL COMMENT 'active/car/public_transport/other',
  screen_time_hours DOUBLE NOT NULL COMMENT 'Horas diárias de tela'  
)
USING DELTA
LOCATION '/mnt/gold/dim_atividade'
COMMENT 'Dimensão com informações sobre atividade física e transporte';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.fact_obesidade (
  id_fato BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'ID sequencial artificial',
  id_paciente STRING NOT NULL COMMENT 'Referência a dim_paciente',
  id_habitos STRING NOT NULL COMMENT 'Referência a dim_habitos',
  id_atividade STRING NOT NULL COMMENT 'Referência a dim_atividade',
  obesity_level STRING NOT NULL COMMENT 'Nível detalhado de obesidade',
  obesity_category STRING COMMENT 'Classificação simplificada',
  processed_date DATE NOT NULL COMMENT 'Data de processamento'
)
USING DELTA
LOCATION '/mnt/gold/fact_obesidade'
COMMENT 'Tabela fato com métricas de obesidade e chaves para dimensões';

In [0]:
%sql
INSERT INTO gold.dim_paciente (
  id_paciente, gender, age, age_group, height_m, weight_kg, bmi, has_family_history
)
SELECT
  patient_key, 
  gender,
  age,
  age_group,
  height_m,
  weight_kg,
  bmi,
  has_family_history
FROM silver.obesity_clean;

num_affected_rows,num_inserted_rows
2087,2087


In [0]:
%sql SELECT * FROM gold.dim_paciente LIMIT 5;  

id_paciente,gender,age,age_group,height_m,weight_kg,bmi,has_family_history
fd4d589c0c96dfd91baca2a2ac2d5001,MALE,18,young_adult,1.7081,51.3147,17.6,true
41c5fd1e097f807578e8021fb1329abd,MALE,21,young_adult,1.7122,98.7000,33.7,true
655411e1adb4032e225aa2195c766dd5,FEMALE,23,young_adult,1.6108,82.5330,31.8,true
74d0b8b0a2862482dd5f4a556a208611,MALE,18,young_adult,1.7387,50.2487,16.6,true
66d6ff722631c2112289d5fc63aacd80,FEMALE,16,adolescent,1.6038,65.0000,25.3,true


In [0]:
%sql
INSERT INTO gold.dim_habitos
SELECT DISTINCT
  md5(concat_ws('|', 
    silver.obesity_clean.calorie_intake_level,
    cast(silver.obesity_clean.vegetable_intake_freq as string),
    cast(silver.obesity_clean.meals_per_day as string),
    silver.obesity_clean.eating_between_meals,
    silver.obesity_clean.alcohol_consumption,
    cast(silver.obesity_clean.is_smoker as string),
    cast(silver.obesity_clean.water_intake as string),
    cast(silver.obesity_clean.monitors_calories as string)
  )) AS id_habitos,
  calorie_intake_level,
  vegetable_intake_freq,
  meals_per_day,
  eating_between_meals,
  alcohol_consumption,
  is_smoker,
  water_intake,
  monitors_calories
FROM silver.obesity_clean
WHERE md5(concat_ws('|', 
    silver.obesity_clean.calorie_intake_level,
    cast(silver.obesity_clean.vegetable_intake_freq as string),
    cast(silver.obesity_clean.meals_per_day as string),
    silver.obesity_clean.eating_between_meals,
    silver.obesity_clean.alcohol_consumption,
    cast(silver.obesity_clean.is_smoker as string),
    cast(silver.obesity_clean.water_intake as string),
    cast(silver.obesity_clean.monitors_calories as string)
  )) 
  NOT IN (SELECT id_habitos FROM gold.dim_habitos);

num_affected_rows,num_inserted_rows
1689,1689


In [0]:
%sql SELECT * FROM gold.dim_habitos LIMIT 5;  

id_habitos,calorie_intake_level,vegetable_intake_freq,meals_per_day,eating_between_meals,alcohol_consumption,is_smoker,water_intake,monitors_calories
6b8824707a724736acf2ed568aa1eda5,high_calorie,2.103335,2,sometimes,non_drinker,false,3.0,false
3c43d92cc2c03b84f0f04210b238fc15,high_calorie,2.694281,3,sometimes,non_drinker,false,2.653831,false
192f247fe6ab23393ae7ac10f578a78c,high_calorie,3.0,3,sometimes,sometimes,false,2.480277,false
2c495699fd0baf3ef03ae4ed583feee1,high_calorie,2.334474,1,sometimes,non_drinker,false,2.776724,false
351cd4181447e2580064367737776079,high_calorie,3.0,3,sometimes,non_drinker,false,2.0,false


In [0]:
%sql
INSERT INTO gold.dim_atividade
SELECT DISTINCT
  md5(concat_ws('|',
    cast(silver.obesity_clean.physical_activity_freq as string),
    silver.obesity_clean.transportation_type,
    cast(silver.obesity_clean.screen_time_hours as string)
  )) AS id_atividade,
  physical_activity_freq,
  transportation_type,
  screen_time_hours
FROM silver.obesity_clean
WHERE md5(concat_ws('|',
    cast(silver.obesity_clean.physical_activity_freq as string),
    silver.obesity_clean.transportation_type,
    cast(silver.obesity_clean.screen_time_hours as string)
  ))
NOT IN (SELECT id_atividade FROM gold.dim_atividade);

num_affected_rows,num_inserted_rows
1532,1532


In [0]:
%sql SELECT * FROM gold.dim_atividade LIMIT 5;  

id_atividade,physical_activity_freq,transportation_type,screen_time_hours
1cda122f8c265f912b4161317186b7ec,0.0,car,0.388271
ccbcf6e15831e77c983ab3829a8bcb4d,0.261274,car,0.0
04b8ffd8640620c2c3c3eb842e9f1930,0.003938,public_transport,0.526999
ef404db962ade14030e5b5626ed3a916,1.876051,public_transport,0.938791
dea4c0115dc5250157a7be58bea14a43,1.629432,car,0.0


In [0]:
%sql
INSERT INTO gold.fact_obesidade (
  id_paciente, 
  id_habitos, 
  id_atividade, 
  obesity_level,
  obesity_category,
  processed_date
)
SELECT 
  p.id_paciente,
  h.id_habitos,
  a.id_atividade,
  s.obesity_level AS obesity_level,
  s.obesity_category AS obesity_category,
  DATE(s.processed_at) AS processed_date
FROM silver.obesity_clean s
JOIN gold.dim_paciente p ON 
  p.id_paciente = s.patient_key
JOIN gold.dim_habitos h ON h.id_habitos = md5(concat_ws('|', 
    s.calorie_intake_level,
    cast(s.vegetable_intake_freq as string),
    cast(s.meals_per_day as string),
    s.eating_between_meals,
    s.alcohol_consumption,
    cast(s.is_smoker as string),
    cast(s.water_intake as string),
    cast(s.monitors_calories as string)
  ))
JOIN gold.dim_atividade a ON a.id_atividade = md5(concat_ws('|', 
    cast(s.physical_activity_freq as string),
    s.transportation_type,
    cast(s.screen_time_hours as string)
  ));

num_affected_rows,num_inserted_rows
2087,2087


In [0]:
%sql
SELECT 
  (SELECT COUNT(*) FROM bronze.obesity_raw) AS bronze_count,
  (SELECT COUNT(*) FROM silver.obesity_clean) AS silver_count,
  (SELECT COUNT(DISTINCT patient_key) FROM silver.obesity_clean) AS unique_patients,
  (SELECT COUNT(DISTINCT md5(concat(calorie_intake_level, cast(vegetable_intake_freq as string), cast(meals_per_day as string), eating_between_meals, alcohol_consumption, cast(is_smoker as string), cast(water_intake as string), cast(monitors_calories as string)))) FROM silver.obesity_clean) AS unique_habitos,
  (SELECT COUNT(DISTINCT md5(concat(cast(physical_activity_freq as string), transportation_type, cast(screen_time_hours as string)))) FROM silver.obesity_clean) AS unique_atividades,
  (SELECT COUNT(*) FROM gold.dim_paciente) AS dim_paciente_count,
  (SELECT COUNT(*) FROM gold.dim_habitos) AS dim_habitos_count,
  (SELECT COUNT(*) FROM gold.dim_atividade) AS dim_atividade_count,
  (SELECT COUNT(*) FROM gold.fact_obesidade) AS fact_count;


bronze_count,silver_count,unique_patients,unique_habitos,unique_atividades,dim_paciente_count,dim_habitos_count,dim_atividade_count,fact_count
2111,2087,2087,1689,1532,2087,1689,1532,2087


In [0]:
%sql
SELECT 
  'Registros na fato' AS metric,
  COUNT(*) AS value
FROM gold.fact_obesidade

UNION ALL

SELECT 
  'Registros sem correspondência em dim_paciente',
  COUNT(*)
FROM gold.fact_obesidade f
LEFT JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
WHERE p.id_paciente IS NULL

UNION ALL

SELECT 
  'Registros sem correspondência em dim_habitos',
  COUNT(*)
FROM gold.fact_obesidade f
LEFT JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
WHERE h.id_habitos IS NULL

UNION ALL

SELECT 
  'Registros sem correspondência em dim_habitos',
  COUNT(*)
FROM gold.fact_obesidade f
LEFT JOIN gold.dim_atividade a ON f.id_atividade = a.id_atividade
WHERE a.id_atividade IS NULL;



metric,value
Registros sem correspondência em dim_paciente,0
Registros sem correspondência em dim_habitos,0
Registros sem correspondência em dim_habitos,0
Registros na fato,2087



### 5.3 Linhagem dos Dados

- **Fonte**: Dataset "Obesity Levels based on Eating Habits and Physical Condition" – [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition).
- **Formato Original**: CSV
- **Camada Bronze**: `bronze.obesity_raw`
- **Transformações Aplicadas**:
  - Remoção de valores inconsistentes e duplicados.
  - Conversão de tipos e categorização de variáveis.
  - Cálculo de campos derivados como `BMI`, `faixa etária`.
  - Geração de IDs hash (chaves técnicas).
- **Ferramentas**: SQL no Databricks, com suporte ao Delta Lake para armazenamento transacional e versionamento dos dados.

---
### 5.4 Catálogo de Dados
#### 📌 Tabela Fato: `gold.fact_obesidade`

| Campo              | Tipo     | Descrição                                                                 |
|-------------------|----------|---------------------------------------------------------------------------|
| `id_fato`          | BIGINT   | Identificador único da linha (auto incrementado)                         |
| `id_paciente`      | STRING   | Chave estrangeira para `dim_paciente`                                    |
| `id_habitos`       | STRING   | Chave estrangeira para `dim_habitos`                                     |
| `id_atividade`     | STRING   | Chave estrangeira para `dim_atividade`                                   |
| `obesity_level`    | STRING   | Categoria detalhada (ex: Obesity_Type_I, Normal_Weight)                  |
| `obesity_category` | STRING   | Categoria simplificada: underweight, normal, overweight, obese           |
| `processed_date`   | DATE     | Data de processamento da linha                                           |

---

#### 👤 Dimensão: `gold.dim_paciente`

| Campo               | Tipo         | Descrição                                                                    | Domínio                          |
|--------------------|--------------|------------------------------------------------------------------------------|----------------------------------|
| `id_paciente`       | STRING       | Hash MD5 dos dados pessoais (chave única)                                   | —                                |
| `gender`            | STRING       | Gênero biológico                                                             | 'MALE', 'FEMALE'                 |
| `age`               | INT          | Idade em anos                                                                | 14 a 100                         |
| `age_group`         | STRING       | Faixa etária                                                                 | adolescent, young_adult, adult, senior |
| `height_m`          | DECIMAL(10,4)| Altura em metros                                                             | 1.20 a 2.50                      |
| `weight_kg`         | DECIMAL(10,4)| Peso em quilogramas                                                          | 30 a 300                         |
| `bmi`               | DECIMAL(4,1) | Índice de Massa Corporal (peso / altura²)                                   | ~10 a ~60                        |
| `has_family_history`| BOOLEAN      | Possui histórico familiar de sobrepeso                                      | TRUE / FALSE                     |

---

#### 🍽️ Dimensão: `gold.dim_habitos`

| Campo                    | Tipo     | Descrição                                                             | Domínio                                    |
|-------------------------|----------|----------------------------------------------------------------------|--------------------------------------------|
| `id_habitos`             | STRING   | Hash MD5 dos hábitos                                                 | —                                          |
| `calorie_intake_level`  | STRING   | Consumo calórico                                                     | low_calorie, high_calorie                  |
| `vegetable_intake_freq` | DOUBLE   | Frequência de consumo de vegetais                                    | 1.0 a 3.0                                  |
| `meals_per_day`         | INT      | Número de refeições principais diárias                               | 1 a 4                                      |
| `eating_between_meals`  | STRING   | Frequência de alimentação entre refeições                            | none, sometimes, frequently, always        |
| `alcohol_consumption`   | STRING   | Frequência de consumo de álcool                                      | non_drinker, sometimes, frequently, always |
| `is_smoker`             | BOOLEAN  | Indica se é fumante                                                  | TRUE / FALSE                               |
| `water_intake`          | DOUBLE   | Consumo diário de água (litros ou frequência)                        | 1.0 a 3.0                                  |
| `monitors_calories`     | BOOLEAN  | Se monitora a ingestão calórica                                      | TRUE / FALSE                               |

---

#### 🏃‍♂️ Dimensão: `gold.dim_atividade`

| Campo                   | Tipo     | Descrição                                              | Domínio                            |
|------------------------|----------|--------------------------------------------------------|------------------------------------|
| `id_atividade`          | STRING   | Hash MD5 dos hábitos de atividade física               | —                                  |
| `physical_activity_freq`| DOUBLE   | Frequência semanal de atividade física                 | 0.0 a 3.0                          |
| `transportation_type`   | STRING   | Tipo principal de transporte utilizado                 | active, car, public_transport, other |
| `screen_time_hours`     | DOUBLE   | Tempo médio diário em frente a telas (em horas)        | 0.0 a 2.0                         |

---

### 📎 Observações Finais

- Todos os identificadores foram gerados com hash `MD5` dos campos naturais para evitar duplicidade e facilitar junções.
- As tabelas foram particionadas e salvas em formato Delta para permitir otimizações de leitura e versionamento.



## 6. Análise Exploratória - Respostas às Perguntas

### 6.1 **Pergunta 1 (Água vs Obesidade): Pessoas que consomem mais água tendem a ter menor nível de obesidade?**

In [0]:
%sql
SELECT 
    f.obesity_level,
    ROUND(AVG(h.water_intake), 2) AS avg_water_intake,
    COUNT(*) AS total_pacientes
FROM gold.fact_obesidade f
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
GROUP BY f.obesity_level
ORDER BY avg_water_intake DESC;

obesity_level,avg_water_intake,total_pacientes
Obesity_Type_III,2.21,324
Obesity_Type_I,2.11,351
Overweight_Level_II,2.03,290
Overweight_Level_I,2.01,276
Obesity_Type_II,1.88,297
Insufficient_Weight,1.88,267
Normal_Weight,1.86,282


Ao observar os dados, percebemos que pacientes que consomem mais água não apresentam menor nível de obesidade — pelo contrário, há uma tendência inversa: indivíduos com níveis mais altos de obesidade relatam, em média, maior consumo de água. Por exemplo:
- Pacientes com **Obesity_Type_III** apresentam o maior consumo médio de água (2.21).
- Pacientes com **Normal_Weight** e **Insufficient_Weight** estão entre os que têm menor média (1.86 e 1.88, respectivamente).

Portanto, à primeira vista, observa-se que o consumo médio de água tende a ser maior entre indivíduos com níveis mais elevados de obesidade, o que pode parecer contraintuitivo. No entanto, essa tendência pode indicar que pessoas obesas estejam em processo de reeducação alimentar ou sob orientação médica, adotando hábitos mais saudáveis — como o aumento da ingestão de água. Por outro lado, indivíduos com peso normal ou insuficiente talvez não percebam a mesma necessidade de alterar esse comportamento.  

Além disso, nota-se um agrupamento intermediário em torno dos casos de sobrepeso e obesidade tipo I, com médias variando entre 2.01 e 2.11, o que reforça a hipótese de uma curva crescente no consumo de água conforme aumenta o nível de obesidade.

### 6.2 **Pergunta 2. (Distribuição de Obesidade por Gênero e Faixa Etária): Qual a distribuição de obesidade por gênero e faixa etária?**

In [0]:
%sql
SELECT 
    p.gender,
    p.age_group,
    f.obesity_category,
    COUNT(*) AS total_pacientes
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
GROUP BY p.gender, p.age_group, f.obesity_category
ORDER BY p.gender, p.age_group, total_pacientes DESC;

gender,age_group,obesity_category,total_pacientes
FEMALE,adolescent,overweight,20
FEMALE,adolescent,underweight,14
FEMALE,adolescent,normal,13
FEMALE,adolescent,obese,3
FEMALE,adult,overweight,69
FEMALE,adult,obese,55
FEMALE,adult,normal,10
FEMALE,adult,underweight,3
FEMALE,senior,overweight,4
FEMALE,senior,obese,2


A análise da distribuição dos níveis de obesidade por gênero e faixa etária revela padrões importantes sobre os grupos mais afetados:

- A faixa **young_adult (18 a 30 anos)** concentra a maior quantidade de pacientes, tanto do sexo feminino quanto masculino, com nível de obesidade:
  - Mulheres **young_adults** obesas: 421 casos
  - Homens **young_adults** obesos: 347 casos
  - Essa faixa também apresenta números expressivos de sobrepeso, indicando uma fase crítica para intervenções de saúde.
- Entre os adolescentes:
  - Meninos apresentam mais casos de baixo peso (underweight, 35 casos), enquanto meninas têm maior incidência de sobrepeso (20 casos).
  - Isso pode indicar diferenças comportamentais ou fisiológicas no desenvolvimento corporal entre os gêneros.
- Nos adultos (31 a 50 anos):
  - Ambos os sexos mantêm altos números de obesidade e sobrepeso, com destaque para os homens adultos obesos (91 casos).
  - Já nas mulheres adultas, o sobrepeso também predomina (69 casos), mas com uma proporção relevante de obesidade (55 casos).
- Em idosos (senior), há menor volume de dados, mas ainda se observa presença significativa de obesidade, especialmente entre homens (48 casos).

Com esses resultados temos que o sexo masculino tende a ter maior incidência de obesidade em faixas etárias mais altas, enquanto o sexo feminino apresenta maior concentração em jovens adultos. Os dados também reforçam a importância de ações de prevenção e conscientização precoce, especialmente entre jovens adultos, que lideram em número absoluto de casos obesos.

### 6.3 **Pergunta 3. (Correlação entre Altura, Peso e Obesidade): Existe uma correlação entre altura, peso e nível de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(p.height_m), 2) AS avg_height,
  ROUND(AVG(p.weight_kg), 2) AS avg_weight,
  ROUND(AVG(p.bmi), 1) AS avg_bmi,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
GROUP BY f.obesity_category
ORDER BY avg_bmi DESC;

obesity_category,avg_height,avg_weight,avg_bmi,count
obese,1.72,109.08,37.0,972
overweight,1.70,78.38,27.1,566
normal,1.68,62.16,22.0,282
underweight,1.69,50.01,17.4,267


Ao analisar os resultados temos os seguintes pontos:
- Quanto maior o peso médio, maior o IMC médio e maior o nível de obesidade — uma correlação direta e esperada.
- Altura média varia muito pouco entre os grupos (de 1.68m a 1.72m), o que mostra que não há uma correlação clara entre altura e obesidade.
- No entanto, pessoas mais altas podem suportar um peso maior sem atingir altos IMCs, o que pode explicar por que a altura média de pacientes obesos ainda é levemente maior que a dos demais grupos.

Portanto, a correlação existente é claramente com o peso. A altura exerce uma influência menor e indireta, mas pode afetar o IMC ao permitir maior peso sem ultrapassar os limiares de obesidade.

### 6.4 **Pergunta 4. (Idade vs Obesidade): Qual a média de idade das pessoas em cada nível de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(p.age), 1) AS avg_age,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
GROUP BY f.obesity_category
ORDER BY avg_age DESC;


obesity_category,avg_age,count
obese,25.4,972
overweight,25.0,566
normal,21.8,282
underweight,19.4,267


Segundo os resultados analisados:
- Pessoas mais jovens tendem a estar na categoria **"Underweight"**  
A idade média dos pacientes com baixo peso é de 19.4 anos, a menor entre todas as categorias. Isso sugere que pacientes em transição entre e fase da adolescenscia para jovens adultos, ainda em fase de crescimento e com metabolismo mais acelerado, estão mais frequentemente nesta categoria.

- A obesidade tende a aumentar com a idade  
A categoria **obese** possui a maior idade média (25.4 anos), indicando que, com o tempo, mudanças no estilo de vida, como piora nos hábitos alimentares ou redução de atividade física, podem contribuir para o ganho de peso.

- Transição gradual do normal para sobrepeso e obesidade  
Pacientes com peso **normal** têm idade média de 21.8 anos, enquanto os **overweight** já atingem 25 anos. Isso mostra uma tendência de ganho gradual de peso a partir da faixa dos 20 anos.

Há uma correlação positiva entre idade e obesidade: quanto maior a idade, maior a tendência de o indivíduo estar nas categorias overweight ou obese. Isso reforça a importância da prevenção e da adoção de hábitos saudáveis desde cedo, especialmente durante a juventude.

### 6.5 **Pergunta 5. (Consumo de Vegetais vs Obesidade): Quem come mais vegetais tem menor índice de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(h.vegetable_intake_freq), 2) AS avg_vegetable_intake,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
GROUP BY f.obesity_category
ORDER BY avg_vegetable_intake DESC;

obesity_category,avg_vegetable_intake,count
obese,2.52,972
underweight,2.47,267
normal,2.34,282
overweight,2.27,566


O consumo de vegetais não parece estar diretamente ligado à redução da obesidade. Os dados mostram que:
- Pessoas obesas (média de 2.52) e underweight (2.47) consomem mais vegetais, em média, do que pessoas com peso normal (2.34) ou sobrepeso (2.27).
- Isso contraria a hipótese de que maior consumo de vegetais estaria relacionado a menores índices de obesidade.

Apesar da tendência estranha dos resultados, temos algumas possíveis explicações para eles: 
- Reeducação alimentar - Pessoas obesas podem estar tentando melhorar sua alimentação, aumentando a ingestão de vegetais.
- Consumo calórico total - Comer vegetais não significa, necessariamente, que a dieta é saudável — outros alimentos calóricos ainda podem estar presentes em excesso.
- Fatores adicionais - Tamanho das porções, sedentarismo, metabolismo e consumo de alimentos industrializados podem impactar mais do que a frequência de vegetais.

Portando, comer mais vegetais isoladamente não garante menor obesidade. Mais à frente, vamos aprofundar essa análise cruzando com o consumo de alimentos calóricos e o nível de atividade física.

### 6.6 **Pergunta 6. (Alimentos Calóricos vs Obesidade): Qual o impacto do consumo frequente de alimentos calóricos na obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  h.calorie_intake_level,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
GROUP BY f.obesity_category, h.calorie_intake_level
ORDER BY f.obesity_category, count DESC;

obesity_category,calorie_intake_level,count
normal,high_calorie,204
normal,low_calorie,78
obese,high_calorie,953
obese,low_calorie,19
overweight,high_calorie,470
overweight,low_calorie,96
underweight,high_calorie,217
underweight,low_calorie,50


De acordo com os resultados:
- **A maioria dos obesos consome alimentos calóricos frequentemente**  
Os dados revelam que 98% das pessoas obesas relatam consumo frequente de alimentos calóricos. Apenas 2% seguem uma alimentação de baixa caloria, indicando uma forte associação entre alta ingestão calórica e níveis elevados de obesidade.
- **Tendência clara: quanto maior o nível de obesidade, maior o consumo calórico**  
Existe um padrão crescente no percentual de pessoas com dieta rica em calorias conforme aumenta a obesidade:
  - Underweight: 81% com alta ingestão calórica
  - Normal: 72% com alta ingestão calórica
  - Overweight: 83%
  - Obese: 98%

- **Pessoas com peso normal ou baixo também consomem alimentos calóricos**  
Apesar da correlação, 204 pessoas com peso normal e 217 com peso abaixo do ideal ainda apresentam consumo elevado de alimentos calóricos. Isso sugere que outros fatores como atividade física, metabolismo e idade também impactam o peso corporal e a obesidade — e o consumo calórico, isoladamente, não explica todos os casos.

### 6.7 **Pergunta 7. (Atividade Física vs Obesidade): Pessoas que fazem mais atividades físicas têm menor nível de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(a.physical_activity_freq), 2) AS avg_activity_freq,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
JOIN gold.dim_atividade a ON f.id_atividade = a.id_atividade
GROUP BY f.obesity_category
ORDER BY avg_activity_freq DESC;

obesity_category,avg_activity_freq,count
underweight,1.27,267
normal,1.25,282
overweight,1.01,566
obese,0.87,972


Sim, os dados indicam que pessoas que praticam mais atividades físicas tendem a ter menor nível de obesidade.
- Pessoas com menor índice de obesidade fazem mais atividades físicas. As categorias **Underweight** e **Normal** apresentam as maiores médias de prática de exercícios físicos, com 1.27 e 1.25 respectivamente.
- Já indivíduos **Overweight** e **Obese** têm médias consideravelmente menores, com 1.01 e 0.87. Isso mostra uma relação inversa entre obesidade e atividade física: quanto menor o nível de obesidade, maior a frequência de exercícios.
- A diferença entre **Underweight** e **Normal** é pequena, mas a queda é significativa ao comparar com o grupo **Obese**.

Essa análise reforça que incentivar a prática de atividades físicas pode ser uma estratégia eficaz para reduzir ou evitar a obesidade.

### 6.8 **Pergunta 8. (Hábitos Saudáveis vs Obesidade): Existe uma relação entre hábitos saudáveis (maior consumo de vegetais, menor consumo de alimentos calóricos e maior atividade física) e os níveis de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(h.vegetable_intake_freq), 2) AS avg_vegetable_intake,
  ROUND(AVG(
    CASE 
      WHEN h.calorie_intake_level = 'low_calorie' THEN 0
      WHEN h.calorie_intake_level = 'high_calorie' THEN 1
    END
  ), 2) AS avg_caloric_food,
  ROUND(AVG(a.physical_activity_freq), 2) AS avg_physical_activity,
  COUNT(*) AS total_pacientes
FROM gold.fact_obesidade f
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
JOIN gold.dim_atividade a ON f.id_atividade = a.id_atividade
GROUP BY f.obesity_category
ORDER BY obesity_category;

obesity_category,avg_vegetable_intake,avg_caloric_food,avg_physical_activity,total_pacientes
normal,2.34,0.72,1.25,282
obese,2.52,0.98,0.87,972
overweight,2.27,0.83,1.01,566
underweight,2.47,0.81,1.27,267


Com base nesses resultados, podemos tirar algumas conclusões interessantes sobre a relação entre hábitos saudáveis e os níveis de obesidade:  
**Pessoas com hábitos mais saudáveis tendem a ter menor obesidade**

- **Consumo de vegetais:** embora pessoas obesas tenham o maior consumo médio (2.52), os indivíduos com peso normal e underweight também apresentam consumo elevado (2.34 e 2.47). Isso indica que só comer vegetais não é suficiente — outros fatores precisam ser considerados.
- **Consumo de alimentos calóricos (avg_caloric_food):**
  - Pessoas obesas: 0.98 → quase todos consomem alta caloria.
  - Pessoas com peso normal: 0.72 → menor consumo calórico médio.
  - Isso mostra uma clara relação entre alta ingestão calórica e obesidade.
- **Atividade física:**
  - Underweight (1.27) e normal (1.25) têm os maiores níveis de atividade.  
  - Obesos (0.87) têm a menor média.
  - Isso confirma que atividade física tem forte influência no controle de peso.



### 6.9 **Pergunta 9. (Refeições Diárias vs Obesidade): Pessoas que comem mais refeições por dia têm menor tendência à obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  ROUND(AVG(h.meals_per_day), 2) AS avg_meals_per_day,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
GROUP BY f.obesity_category
ORDER BY avg_meals_per_day DESC;

obesity_category,avg_meals_per_day,count
normal,2.75,282
underweight,2.7,267
obese,2.57,972
overweight,2.29,566


- Pessoas com peso normal ou abaixo do peso fazem mais refeições por dia
  - Comem em média 2.75 refeições/dia, enquanto underweight comem 2.70 refeições/dia.
- Pessoas obesas fazem menos refeições diárias
  - A média de refeições diárias dos obesos é 2.57, menor que a dos normais e underweight.
- Os overweight comem ainda menos, com 2.29 refeições/dia.

Portanto, fracionar mais as refeições ao longo do dia pode estar relacionado a menores índices de obesidade, enquanto fazer menos refeições pode contribuir para o ganho de peso. Isso porque, ao comer menos vezes, as pessoas tendem a consumir mais calorias por refeição ou fazer escolhas alimentares menos saudáveis. Por outro lado, comer com mais frequência pode ajudar a controlar melhor a fome e evitar excessos, funcionando como uma estratégia auxiliar no controle de peso.

### 6.10 **Pergunta 10. (Consumo de Álcool vs Obesidade): Há uma relação entre o consumo de álcool e os níveis de obesidade?**

In [0]:
%sql
SELECT 
  f.obesity_category,
  h.alcohol_consumption,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_paciente p ON f.id_paciente = p.id_paciente
JOIN gold.dim_habitos h ON f.id_habitos = h.id_habitos
GROUP BY f.obesity_category, h.alcohol_consumption
ORDER BY f.obesity_category, count DESC;

obesity_category,alcohol_consumption,count
normal,sometimes,159
normal,non_drinker,104
normal,frequently,18
normal,always,1
obese,sometimes,719
obese,non_drinker,237
obese,frequently,16
overweight,sometimes,353
overweight,non_drinker,178
overweight,frequently,35


Com base nos dados, não parece haver uma relação direta entre o consumo de álcool e os níveis de obesidade. A maioria das pessoas em todas as categorias de peso relatam consumir álcool “às vezes”, independentemente do nível de obesidade. Por exemplo:

- **Obese**: 719 (às vezes), 237 (não bebem)
- **Normal**: 159 (às vezes), 104 (não bebem)
- **Overweight**: 353 (às vezes), 178 (não bebem)
- **Underweight**: 149 (às vezes), 117 (não bebem)

A proporção entre consumidores ocasionais e não consumidores se mantém relativamente constante entre os grupos. Casos de consumo frequente ou diário são raros em todos os níveis, indicando que esse comportamento não é predominante na nossa base de dados.


### 6.11 **Pergunta 11. (Transporte vs Obesidade): Quem usa transporte ativo (bicicleta, caminhada) tem menores índices de obesidade do que aqueles que usam carro ou transporte público?**

In [0]:
%sql
SELECT 
  a.transportation_type,
  f.obesity_category,
  COUNT(*) AS count
FROM gold.fact_obesidade f
JOIN gold.dim_atividade a ON f.id_atividade = a.id_atividade
GROUP BY a.transportation_type, f.obesity_category
ORDER BY a.transportation_type, f.obesity_category;


transportation_type,obesity_category,count
active,normal,35
active,obese,4
active,overweight,17
active,underweight,6
car,normal,44
car,obese,206
car,overweight,160
car,underweight,46
other,normal,6
other,obese,3


Sim, os dados sugerem que quem utiliza transporte ativo (como caminhada ou bicicleta) apresenta menores índices de obesidade.
- Entre os que usam transporte ativo, apenas 4 são obesos, enquanto 35 têm peso normal, e os demais estão entre underweight e overweight. Ou seja, a maioria está fora das categorias mais altas de obesidade.
- Já entre os que usam carro, a quantidade de obesos é bem maior (206), superando inclusive o número de pessoas com peso normal (44). O mesmo padrão se repete no transporte público, onde há 759 obesos contra 197 com peso normal.

Ou seja, há uma tendência clara:
- Usuários de transporte ativo têm menor prevalência de obesidade.
- Usuários de carro ou transporte público apresentam maiores índices de obesidade.

Isso indica que o deslocamento ativo contribui para o gasto calórico diário e ajuda no controle do peso corporal.

## 7. Conclusão

A análise exploratória realizada permite identificar algumas evidências importantes:

- **Mudança de hábitos isolada pode não ser suficiente.**  
O consumo de água, a ingestão de vegetais, a redução de alimentos calóricos e a prática de atividades físicas precisam ocorrer em conjunto para gerar um impacto significativo na redução dos níveis de obesidade.
- **Jovens adultos merecem atenção especial.**  
Observamos um alto índice de obesidade na faixa etária dos 20 a 25 anos, o que indica a necessidade de políticas de prevenção mais direcionadas para essa fase da vida.
- **O consumo de álcool tem relação mais forte com sobrepeso.**  
Embora presente em todas as categorias, o álcool parece influenciar mais o ganho moderado de peso do que os casos de obesidade severa.
- **Transporte ativo é aliado da saúde.**  
Indivíduos que utilizam meios de transporte ativos, como bicicleta ou caminhada, apresentam menor incidência de obesidade, reforçando a importância de incentivar a mobilidade ativa nas cidades.

Quanto ao **pipeline de dados**, sua construção possibilitou uma análise robusta, com alto nível de integridade e organização dos dados, por meio das camadas Bronze, Silver e Gold. A modelagem em estrela e a separação das dimensões facilitaram a extração de insights. Para trabalhos futuros, será interessante explorar novas métricas, desenvolver mais perguntas analíticas e aprofundar a correlação entre variáveis, potencializando ainda mais os resultados obtidos.

## 8. Auto Avaliação

Na minha opinião, a autoavaliação do trabalho é bastante positiva. Foi possivel atingir o principal objetivo do projeto, que era estruturar um pipeline de dados eficiente e utilizá-lo para realizar uma análise exploratória relevante sobre obesidade e hábitos de vida. As perguntas propostas foram bem respondidas com base nos dados tratados, e os insights obtidos são coerentes e úteis tanto para fins acadêmicos quanto para possíveis aplicações em políticas de saúde. Por fim, esse projeto demonstrou uma boa organização, uso adequado das ferramentas e aplicação correta dos conceitos de engenharia de dados.

